<a href="https://colab.research.google.com/github/BehzadBarati/Ingredient-Maps/blob/main/Food_Recipes_RecipeNLG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Behzad Barati

Abstract:

*   This notebook produces SQL tables for working with almost 500K recipies of RecipeNLG dataset.
*   RecipeNLG dataset is composed of Recipe1M dataset and other recipes which were added by RecipeNLG authors.
*   We load RecipeNLG dataset, but For better quality of data, I removed recipes which were gathered by RecipeNLG authors. So, generated SQL tables are dealing with Recipe1M data only.
*   Some preprocessing actions also took place on dataset (in Preprocessing recipe_table section). so we ended up with almost 500K recipes.
___
Source:

My main refrences are [RecipeNLG paper](https://www.aclweb.org/anthology/2020.inlg-1.4.pdf) and its [dataset](https://recipenlg.cs.put.poznan.pl).
___
Input: 

1- Dataset of [RecipeNLG](https://recipenlg.cs.put.poznan.pl)

Ouput:

1- EDA report on RecipeNLG dataset (including "EDA-Report-RecipeNLG.html" file, word cloud pictures and list of source websites of recipies as "Websites-RecipeNLG.csv" file)

2- recipe_table

3- ner_table

4- step_table

5- recipe_ner_associate_table

6- recipe_step_associate_table
___
Hints:

1_ As our csv file is greater than 2 gigabytes, I prefer to use cloud services(here google colab). I uploaded RecipeNLG dataset in my [google drive](https://drive.google.com/drive/folders/1g1ZNYKlLN4hyP8ywHXWa2Iu1oQ4wxSgR?usp=sharing). It is public.


2_ If there is out of memory error in running "ProfileReport", please first re-install latest version of "pandas_profiling" library, then try "minimal=True" argument in "profileReport" for eliminating some calculations. (pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip)

# Import needed libraries

In [1]:
# Install pandas_profiling library
# pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

import numpy as np
import pandas as pd
from wordcloud import WordCloud             # Make wordcloud pictures
from pandas_profiling import ProfileReport  # Generate brief report on our dataframe
import matplotlib.pyplot as plt
from google.colab import drive              # Mount google drive to colab notebook
import re                                   
import string                               # removing special characters
from pandas.core.common import flatten      # to make nested lists flat

# Load data

In [2]:
# Mount google drive to colab notebook
# Our dataset will be read as recipe_table.

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
cd gdrive/MyDrive/Projects/Ingredient-Maps/Phase1

/content/gdrive/MyDrive/Projects/Ingredient-Maps/Phase1


In [4]:
# Reading file and check if data is loaded

recipe_table = pd.read_csv('./dataset/RecipeNLG.csv')
print('Number of recipes in dataset: ', len(recipe_table))
recipe_table.rename(columns={'Unnamed: 0': 'id', 'title': 'tag_value', 'directions': 'steps', 'NER': 'ner'}, inplace=True)
print('last 5 recipes:')
recipe_table.tail(5)

Number of recipes in dataset:  2231142
last 5 recipes:


,id,tag_value,ingredients,steps,link,source,ner
2231137,2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."
2231141,2231141,Polpette in Spicy Tomato Sauce,"[""1 pound ground veal"", ""1/2 pound sweet Itali...","[""Preheat the oven to 350."", ""In a bowl, mix t...",www.foodandwine.com/recipes/polpette-spicy-tom...,Recipes1M,"[""ground veal"", ""sausage"", ""bread crumbs"", ""mi..."


In [5]:
recipe_table.head()

,id,tag_value,ingredients,steps,link,source,ner
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


# EDA (Exploratory Data Analysis)
I dont want to generate report every time I run it, so EDA, world cloud and website list cells are muted.

In [ ]:
'''
# Column 'id' seems to be useless for EDA section,so I drop it for making our dataset smaller.

recipe_table.drop('id', axis='columns', inplace=True)
'''

"\n# Column 'ID' seems to be useless for EDA section,so I drop it for making our dataset smaller.\n\nrecipe_table.drop('ID', axis='columns', inplace=True)\n"

In [ ]:
'''
# Generate a quick report from our dataset 

profile = ProfileReport(recipe_table, minimal=True)
profile.to_file("EDA-Report-RecipeNLG.html")
profile
'''

'\n# Generate a quick report from our dataset \n\nprofile = ProfileReport(recipe_table, minimal=True)\nprofile.to_file("EDA-Report-RecipeNLG.html")\nprofile\n'

## World clouds

In [ ]:
'''
# For creating word clouds, I used WordCloud library which was imported before.

def minimal_wordcloud(df, column):
    """
    Generate a simple wordcloud similar to: 
    https://www.kaggle.com/paultimothymooney/explore-recipe-nlg-dataset/data.
    The only import required is: from wordcloud import WordCloud
    """
    text = str(df[column].values)
    wordcloud = WordCloud().generate(text)
    image = wordcloud.to_image()
    plt.axis("off")
    plt.imshow(image)
    plt.show()
'''

'\n# For creating word clouds, I used WordCloud library which was imported before.\n\ndef minimal_wordcloud(df, column):\n    """\n    Generate a simple wordcloud similar to: \n    https://www.kaggle.com/paultimothymooney/explore-recipe-nlg-dataset/data.\n    The only import required is: from wordcloud import WordCloud\n    """\n    text = str(df[column].values)\n    wordcloud = WordCloud().generate(text)\n    image = wordcloud.to_image()\n    plt.axis("off")\n    plt.imshow(image)\n    plt.show()\n'

In [ ]:
'''
# Print word clouds

for c in recipe_table.columns:
    print('\nworld cloud of contents in column {}'.format(c))
    minimal_wordcloud(recipe_table, c)
'''

"\n# Print word clouds\n\nfor c in recipe_table.columns:\n    print('\nworld cloud of contents in column {}'.format(c))\n    minimal_wordcloud(recipe_table, c)\n"

## list of websites in RecipieNLG

In [ ]:
'''
# Based on link column I tried to extract website names.
# This func helps to select website names where we do not have 'www' at beginning

func = lambda x: x[1] if x[0] == 'www' else x[0]
recipe_table['website'] = recipe_table['link'].str.split('.').apply(func)
'''

"\n# Based on link column I tried to extract website names.\n# This func helps to select website names where we do not have 'www' at beginning\n\nfunc = lambda x: x[1] if x[0] == 'www' else x[0]\nrecipe_table['website'] = recipe_table['link'].str.split('.').apply(func)\n"

In [ ]:
'''
recipe_table['website'].value_counts().rename_axis('websites').to_csv('./reports/Websites-RecipeNLG.csv',  header=['No. of recipies'])
recipe_table['website'].value_counts().rename_axis('websites')
'''

"\nrecipe_table['website'].value_counts().rename_axis('websites').to_csv('./reports/Websites-RecipeNLG.csv',  header=['No. of recipies'])\nrecipe_table['website'].value_counts().rename_axis('websites')\n"

# Prprocess recipe_tables

In [6]:
# filter Recipes1M recipes as it seems they have better quality, comapring recipes which their source mentioned as 'Gathered'

recipe_table = recipe_table[recipe_table['source'] == 'Recipes1M']
recipe_table.drop(columns=['ingredients', 'link', 'source'], inplace=True)
recipe_table['tag'] = 'title'

## preprocess ner column 

In [7]:
# make characters lowercase strictly.

recipe_table['ner'] = recipe_table['ner'].str.casefold()
recipe_table['tag_value'] = recipe_table['tag_value'].str.casefold()

In [8]:
# remove punctuations from tag_value and ner columns

punctuations = str.maketrans('', '', '!"#$%&\'()*+-./:;<=>?@[\\]^_`{|}~')
recipe_table['ner'] = recipe_table['ner'].str.translate(punctuations)
recipe_table['tag_value'] = recipe_table['tag_value'].str.translate(punctuations)

In [9]:
# drop rows with no tag_value or ner

recipe_table = recipe_table[recipe_table['tag_value'].notna()]
recipe_table = recipe_table[recipe_table['ner'].notna()]
recipe_table = recipe_table[recipe_table['ner'] != '']

In [10]:
# split ner components to make a list out of them.

recipe_table['ner'] = recipe_table['ner'].str.split(',')

In [11]:
# remove spaces before/after items of list

recipe_table['ner'] = [[val.strip() for val in sublist] for sublist in recipe_table['ner'].values]

In [12]:
# I noticed some ner start with "a " (i.e. a milk). so we should remove them.

recipe_table['ner'] = [[re.sub('^a ', '', val) for val in sublist] for sublist in recipe_table['ner'].values]

In [13]:
# remove spaces before/after items of list once again

recipe_table['ner'] = [[val.strip() for val in sublist] for sublist in recipe_table['ner'].values]

In [14]:
# remove empty items from lists in ner column

recipe_table['ner'] = recipe_table['ner'].apply(lambda row: list(filter(None, row)))

In [15]:
# remove items from lists in ner column which are only 1 character (i.e. 'm')

recipe_table['ner'] = recipe_table['ner'].apply(lambda row: [item for item in row if len(item) > 1] )

In [16]:
# remove duplicates items in each row of ner column

recipe_table['ner'] = recipe_table['ner'].apply(lambda row: list(set(row)))

In [17]:
# romve recipes where they have less than two ners.

recipe_table = recipe_table[recipe_table['ner'].str.len() > 1]

## preprocess steps column

In [18]:
# split recipe_table['steps'] to make a list out of each record.
# since in some steps there are comma, we can not split steps based on it.
# so we split based on " and then remove items which are meaningless (less than 4 charachters)

recipe_table['steps'] = recipe_table['steps'].str.split('"')
recipe_table['steps'] = recipe_table['steps'].apply(lambda row: [item for item in row if len(item) > 3] )

In [19]:
# some steps consists multiple sentences which are ended with dot. so we split steps again based on dot.
# by runing split('.), we will get some two dimensional lists and also some blank spaces as items
# so we make steps lists flat and remove items with less 2 characters.
# remove spaces before/after items of list

recipe_table['steps'] = [[val.split('.') for val in sublist] for sublist in recipe_table['steps'].values]
recipe_table['steps'] = recipe_table['steps'].apply(lambda row: list(flatten(row)))
recipe_table['steps'] = recipe_table['steps'].apply(lambda row: [item for item in row if len(item) > 2])
recipe_table['steps'] = [[val.strip() for val in sublist] for sublist in recipe_table['steps'].values]

In [20]:
# romve recipes where they have less than two steps.

recipe_table = recipe_table[recipe_table['steps'].str.len() > 1]

In [21]:
# reset index due to deleting some rows of recipe_table in previous cells. 
# naming convention for recipe_id column

recipe_table.reset_index(inplace=True, drop=True)
recipe_table['id'] = "rec_" + recipe_table.index.astype(str)

In [ ]:
'''
# since we have a limit of available RAM. we can preprocess original dataframe and then save it as csv, then clean the RAM and once again load it.

recipe_table.to_csv('./dataset/ProcessedRecipeNLG.csv', index=False)
'''

# Build ner, step and associated tables

In [ ]:
'''
# per comments in last cell, since we have a limit of available RAM. we preprocess original dataframe and then save it as csv.
# after clearing RAM, we load it here again for creating ner_table and step_table.
# Mount google drive to colab notebook
# Our dataset will be read as recipe_table.

drive.mount('/content/gdrive', force_remount=True)
recipe_table = pd.read_csv('./MyDrive/Projects/Ingredient-Maps/Phase1/dataset/ProcessedRecipeNLG.csv')

# we convert steps and ner column contents from string to list format. 
import ast
ast_func = lambda a: ast.literal_eval(a)
recipe_table['ner'] = recipe_table['ner'].map(ast_func)
recipe_table['steps'] = recipe_table['steps'].map(ast_func)
'''

## ner_table

In [22]:
# create ner_table and add id column

ner_table = pd.DataFrame(recipe_table['ner'].explode().unique(), columns=['tag_value'])
ner_table['id'] =  ['ner_'+str(i) for i in range(len(ner_table['tag_value']))]
ner_table['tag'] = 'ner'

In [23]:
# set index to ner and make conv dictionary out of ner_table
# by help of conv dictionary, we replace names in recipe_table['ner'] with ids

ner_table.set_index(['tag_value'], inplace=True)
conv = ner_table.to_dict('dict')
recipe_table['ner'] = recipe_table['ner'].apply(lambda row: [conv['id'][v] for v in row if conv['id'].get(v)])
ner_table.reset_index(inplace=True)

In [24]:
'''
# If we want to have all recipes of each ner as a list, we can run this cell.

# add a column to keep recipe_ids and reset index in ordet to fill recipe_id column
# we fill recipe_id column of ner_table with all recipe_ids used ner in. (~ 4 minutes in colab with 8 GB RAM)

ner_table['recipe_id'] = np.empty((len(ner_table), 0)).tolist()
ner_table = ner_table.reset_index().set_index('id')

for k in range(len(recipe_table)):
    for m in recipe_table['ner'][k]:
        ner_table['recipe_id'][m].append(recipe_table['id'][k])


ner_table['no_recipe'] = ner_table['recipe_id'].str.len()
ner_table = ner_table.reset_index()
ner_table = ner_table[['id', 'tag_value', 'no_recipe']]
ner_table.to_csv('./dataset/ner_no_recipe2.csv', index=False)

# this is another approach to fill recipe_id column in ner_table. but this is slow. (~ 100 hours for this problem)
import time

ner_table['recipe_id'] = np.empty((len(ner_table), 0)).tolist()
for i in range(len(ner_table)):
    t0= time.clock()
    for j in range(len(recipe_table)):
        if ner_table['id'][i] in recipe_table['ner'][j]:
           ner_table['recipe_id'][i].append(recipe_table['recipe_id'][j])
    print(time.clock() - t0)
'''

## step_table

In [ ]:
# create step_table and add step_id column

step_table = pd.DataFrame(recipe_table['steps'].explode().unique(), columns=['tag_value'])
step_table['id'] =  ['ste_'+str(i) for i in range(len(step_table['tag_value']))]
step_table['tag'] = 'step'

In [ ]:
# set index to tag_value and make conv dictionary out of step_table
# by help of conv dictionary, we replace names in recipe_table['steps'] with ids

step_table.set_index(['tag_value'], inplace=True)
conv = step_table.to_dict('dict')
recipe_table['steps'] = recipe_table['steps'].apply(lambda row: [conv['id'][v] for v in row if conv['id'].get(v)])
step_table.reset_index(inplace=True)

## Associate Tables

In [ ]:
# creat associate tables between recipe, ner and step tables

recipe_ner_associate_table = recipe_table.explode('ner')
recipe_step_associate_table = recipe_table.explode('steps')

In [ ]:
# keep just necessary columns of main tables

recipe_table = recipe_table[['id', 'tag', 'tag_value']]
ner_table = ner_table[['id', 'tag', 'tag_value']]
step_table = step_table[['id', 'tag', 'tag_value']]

In [ ]:
# keep just necessary columns of associate tables

recipe_ner_associate_table = recipe_ner_associate_table[['id', 'ner']]
recipe_step_associate_table = recipe_step_associate_table[['id', 'steps']]

## Final tables and their heads

In [ ]:
print(len(recipe_table))
recipe_table.head()

576961


,ID,tag,tag_value
0,REC_0,title,worlds best mac and cheese
1,REC_1,title,dilly macaroni salad recipe
2,REC_2,title,gazpacho
3,REC_3,title,kombu tea grilled chicken thigh
4,REC_4,title,zucchini nut bread


In [ ]:
print(len(step_table))
step_table.head()

4797549


,ID,tag,tag_value
0,STE_0,step,Preheat the oven to 350 F
1,STE_1,step,Butter or oil an 8-inch baking dish
2,STE_2,step,Cook the penne 2 minutes less than package dir...
3,STE_3,step,(It will finish cooking in the oven
4,STE_4,step,Rinse the pasta in cold water and set aside


In [ ]:
print(len(ner_table))
ner_table.head()

91662


,ID,tag,tag_value
0,NER_0,ner,milk
1,NER_1,ner,chipotle chili powder
2,NER_2,ner,garlic
3,NER_3,ner,cheddar
4,NER_4,ner,flour


In [ ]:
print(len(recipe_ner_associate_table))
recipe_ner_associate_table.head()

5022648


,ID,ner
0,REC_0,NER_0
0,REC_0,NER_1
0,REC_0,NER_2
0,REC_0,NER_3
0,REC_0,NER_4


In [ ]:
print(len(recipe_step_associate_table))
recipe_step_associate_table.head()

6205935


,ID,steps
0,REC_0,STE_0
0,REC_0,STE_1
0,REC_0,STE_2
0,REC_0,STE_3
0,REC_0,STE_4


# Build SQL tables

In [ ]:
import sqlite3
conn = sqlite3.connect('recipe.db')

In [ ]:
recipe_ner_associate_table.to_sql('recipe_ner_associate_table', conn, index=False)
recipe_step_associate_table.to_sql('recipe_step_associate_table', conn, index=False)
recipe_table.to_sql('recipe_table', conn, index=False)
ner_table.to_sql('ner_table', conn, index=False)
step_table.to_sql('step_table', conn, index=False)

In [ ]:
# check if our tables are in recipe.db

sql = '''SELECT name FROM sqlite_master WHERE type='table';'''
pd.read_sql(sql, conn)

,name
0,recipe_ner_associate_table
1,recipe_step_associate_table
2,recipe_table
3,ner_table
4,step_table


In [ ]:
# check if tables have content.

sql = '''SELECT * FROM recipe_table r WHERE r.id == 'rec_1';'''
pd.read_sql(sql, conn)

,ID,tag,tag_value
0,REC_1,title,dilly macaroni salad recipe
